# Recipes PCA
### By Brian Kitano

Okay, I'm going to use the Epicurious dataset to identify palates common across their recipes. 

## Introduction / Hypothesis

## Materials

## Procedure
1. Download the JSON data
2. Parse the JSON to extract recipe names and ingredients with their quantities.
3. Create the data matrix M, where each column is a recipe and each row is an ingredient; the entry is the quantity in a normalized and standardized quantity (grams?)
4. PCA

Bonus: construct the bipartite graph of ingredients to recipes, and then project it down onto a unipartite graph of ingredients where the weight of each edge is the frequency of connections. 

In [25]:
# 1. Parse the ingredients to extract recipe names and ingredients with their quantities
import json

# load in the epicurious set
with open('full_format_recipes.json') as f:
    data = json.load(f)
    f.close()
    
# print data[0]['ingredients']
# len(data) = 20130

# need to filter out the recipes that don't have ingredients listed
# extractIngredients = lambda i: data[i].keys


In [47]:
def containsIngredients(index):
    if 'ingredients' in data[i].keys():
        return i
    else:
        return 0

# get all of the indices which contain ingredients
cleanedIndices = [ containsIngredients(i) for i in range(len(data)) ]

# get all of the ingredient lists as a list of lists
cleanedIngredientsLists = [ data[i]['ingredients'] for i in cleanedIndices ]

# flatten this list, which might contain duplicates
cleanedIngredients = [ ingredient for ingredientList in cleanedIngredientsLists for ingredient in ingredientList]
print len(cleanedIngredients)

# remove any duplicates
uniqueCleanedIngredients = list(set(cleanedIngredients))
print len(uniqueCleanedIngredients)

199315
83465


In [49]:
# write the ingredients to a file, which we'll then feed to a model
with open('ingredientsList.txt', 'a') as the_file:
    for ingredient in uniqueCleanedIngredients:
        asciiOnlyIngredient = "".join(i for i in ingredient if ord(i)<128)
        ingredientString = asciiOnlyIngredient + "\n"
        the_file.write(ingredientString)


In [50]:
# okay, the model ran and i've got the sauce

# load in the labeled stuff
with open('results.json') as g:
    labeledIngredients = json.load(g)
    g.close()
    
print labeledIngredients[0]

{u'comment': u'fresh', u'name': u'lemon juice', u'qty': u'1 1/4', u'input': u'1 1/4 cups fresh lemon juice', u'display': u"<span class='qty'>1 1/4</span><span class='unit'>cups</span><span class='comment'>fresh</span><span class='name'>lemon juice</span>", u'unit': u'cup'}
